## Checkpoints
- [ ] ``load -> save -> compare``
- [ ] ``make pkg from scratch``
- [ ] ``layers generator``

In [79]:
!python3.10 -m pip install --upgrade pip
!pip install graph-notebook
!pip install graphviz
!pip install pydot
!pip install networkx
!pip install pyvis
!pip install platform
!pip install hashlib
!pip install jycm
%load_ext line_profiler

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be fou

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: Could not find a version that satisfies the requirement platform (from versions: none)
ERROR: No matching distribution found for platform
ERROR: Ignored the following yanked versions: 20081119
ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)
ERROR: No

In [80]:
import os
import matplotlib.pyplot as plt
import networkx as nx
import json
import pprint
import numpy as np
import time
import platform
import hashlib
from enum import Enum
from pyvis.network import Network
from IPython.display import display, HTML
import copy
from jycm.jycm import YouchamaJsonDiffer

In [81]:

class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):

            return str(obj)

        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)

        elif isinstance(obj, (np.complex_, np.complex64, np.complex128)):
            return {'real': obj.real, 'imag': obj.imag}

        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()

        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None

        return json.JSONEncoder.default(self, obj)

class ObjectIdType(Enum):
    Unknown = 'U'
    Link = 'L'
    Timer = 'T'
    Event = 'E'
    Core = 'C'
    Branch = 'B'
    Other = 'O'
    File = 'F'
    Neuron = 'N'
    

class MgennComon:
    nextid = 0
    @staticmethod
    def getNodeName():
        strName = platform.machine() + platform.version() + str(platform.uname()) + platform.processor()
        m = hashlib.sha256()
        m.update(strName.encode('utf-8'))
        nameHash = m.hexdigest()
        return nameHash
    
    @staticmethod
    def getLocalId():
        MgennComon.nextid = MgennComon.nextid + 1
        return np.int64(MgennComon.nextid)
    
    @staticmethod
    def makeId(objPrefix: str):
        MgennComon.nextid = MgennComon.nextid + 1
        return "L%s_%s_%s.%d.%d" % (objPrefix, "PYMGENN", MgennComon.getNodeName(), round(time.time() * 1000), MgennComon.nextid)

class ObjectFactory:
    @staticmethod
    def makeNeuron(peak: float, leak: float, currentEnergy: float = 0, id: np.int64 = 0, receivers: list = []):
        n = {
            "currentEnergy": currentEnergy,
            "energyLeak": leak,
            "mode": "shared",
            "peakEnergy": peak,
            "receivers": []
        }
        for rids in receivers:
            n["receivers"].append(np.int64(rids))
        if id == 0:
            id = MgennComon.getLocalId()
            
        n["id"] = id
        return n

class PackageUtils:
    @staticmethod
    def checkPkg(p: dict):
        if "meta" not in p or not p["meta"]:
            raise ValueError("no meta")
        if "content" not in p or not p["content"]:
            raise ValueError("no content")
        if "contenttype" not in p or p["contenttype"] != "json":
            raise ValueError("no contenttype")
        c = p["content"]
        meta = p["meta"]
        if "tick" not in c or np.int64(c["tick"]) < 0:
            raise ValueError("no tick")    
        for req in ["generation", "history", "id", "parent", "seq", "storage", "inputs", "outputs"]:
            if req not in c:
                raise ValueError("no %s in content" % req)
        for req in ["links", "neurons"]:
            if req not in c["storage"]:
                raise ValueError("no %s in storage" % req)
                
        if c["parent"] != meta["parentSnapshot"]:
            raise ValueError("meta[parent] does not match with snapshot! %s != %s" % (c["parent"], meta["parentSnapshot"]))
        if c["id"] != meta["name"]:
            raise ValueError("meta[id] does not match with snapshot! %s != %s" % (c["id"], meta["name"]))
        print("pkg ok")

        
class Package:
    def __init__(self):
        self.pkg = {}
        self.inputs = []
        self.outputs = []
        self.links = []
        self.neurons = []
        self.state = "ready"
        self.meta = {}
        self.history = {}
        self.external = {}
        self.tick = np.int64(0)
        self.generation = 0
        self.seq = 0
        self.snapshot_id = ""
        self.parent = ""
    
    def __contains__(self, key):
        return self.isLink(key) or self.isNeuron(key) or self.isOutput(key)
    
    def clone(self):
        return copy.deepcopy(self)
    

            
    def findLink(self, id) -> int:
        for i in range(len(self.links)):
            if self.links[i]['id'] == np.int64(id):
                return i
        return -1
    
    def findNeuron(self, id) -> int:
        for i in range(len(self.neurons)):
            if self.neurons[i]['id'] == np.int64(id):
                return i
        return -1
    
    def findOutput(self, id) -> int:
        for i in range(len(self.neurons)):
            if self.neurons[i]['id'] == np.int64(id):
                return i
        return -1
    
    def findInput(self, name) -> int:
        for i in range(len(self.inputs)):
            if self.inputs[i]['id'] == name:
                return i
        return -1
    
    def isLink(self, id) -> bool:
        return (self.findLink(id) >= 0)
    def isNeuron(self, id) -> bool:
        return (self.findNeuron(id) >= 0)
    def isOutput(self, id) -> bool:
        return (self.findOutput(id) >= 0)
    
    def loadFile(self, fname):
        f = open(fname)
        self.pkg = json.load(f)
        f.close()
        PackageUtils.checkPkg(self.pkg)
        
        self.inputs = self.pkg["content"]["inputs"]
        self.outputs = self.pkg["content"]["outputs"]
        self.links = self.pkg["content"]["storage"]["links"]
        self.neurons = self.pkg["content"]["storage"]["neurons"]
        self.history = {}
        self.external = {}
        content = self.pkg["content"]
        self.tick =  np.int64(content["tick"])
        self.generation = int(content["generation"])
        self.seq = int(content["seq"])
        self.snapshot_id = content["id"]
        self.parent = content["parent"]
        if "state" in self.pkg:
            self.state = self.pkg["state"]
        if "meta" in self.pkg:
            self.meta = self.pkg["meta"]
        if "history" in content:
            self.history = content["history"]
        if "external" in content:
            self.external = content["external"]    
            
        # fix types
        for i in range(0, len(self.neurons)):
            self.neurons[i]['id'] = np.int64(self.neurons[i]['id'])
            for j in range(0, len(self.neurons[i]["receivers"])):
                self.neurons[i]["receivers"][j] = np.int64(self.neurons[i]["receivers"][j])
        
        for i in range(0, len(self.inputs)):
            for j in range(0, len(self.inputs[i]["receivers"])):
                ##print(self.inputs[i]["receivers"])
                self.inputs[i]["receivers"][j] = np.int64(self.inputs[i]["receivers"][j])    
        
        for i in range(0, len(self.outputs)):
            self.outputs[i]['id'] = np.int64(self.outputs[i]['id'])
            
        for i in range(0, len(self.links)):
            self.links[i]['id'] = np.int64(self.links[i]['id'])
            self.links[i]['receiverId'] = np.int64(self.links[i]['receiverId'])

    def dump(self):
        pkg = {}
        lastName = self.meta["name"]
        self.meta["branchSeq"] = self.meta["branchSeq"] + 1
        self.meta["parentSnapshot"] = lastName
        self.meta["name"] = self.snapshot_id
        pkgName = "%s_rev0.pkg" % self.meta["name"]
        
        pkg["meta"] = self.meta
        pkg["state"] = self.state
        pkg["contenttype"] = "json"

        content = {}
        content["parent"] = lastName
        content["tick"] = self.tick
        content["id"] = self.snapshot_id
        content["generation"] = self.generation
        content["seq"] = self.seq
        content["inputs"] = self.inputs
        content["outputs"] = self.outputs
        content["storage"] = {}
        content["storage"]["links"] = self.links
        content["storage"]["neurons"] = self.neurons
        content["history"] = self.history
        content["external"] = self.external
        pkg["content"] = content
        return (pkgName, pkg)
    
    def saveToFile(self, fname = "")-> bool:
        pkgname, data = self.dump()
        if not data:
            print("no data!")
            return False
        if not fname:
            fname = pkgname
        with open(fname, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4, sort_keys = True, cls=NumpyEncoder)
            return True
        
    def counts(self):
        i = len(self.inputs)
        o = len(self.outputs)
        l = len(self.links)
        n = len(self.neurons)
        print("inputs:%d, outputs:%d, neurons:%d, links:%d, total:%d" % (i,o,l,n, (i+o+l+n)))
    
    def maxId(self):
        ret = np.int64(0)
        for o in self.outputs:
            ret = max(ret, np.int64(o['id']))
        for n in self.neurons:
            ret = max(ret, np.int64(n['id']))    
        for l in self.links:
            ret = max(ret, np.int64(l['id']))      
        return ret
    
    def nextId(self):
        return self.maxId() + 1
    
    def linkEnds(self, link):
        link_id = int(link["id"])
        #print("looking for ", link_id)
        start = 0
        stop = np.int64(link["receiverId"])
        for n in self.neurons:
            nid = (n["id"])
            ##print(np.array(n["receivers"]))
            if link_id in np.array(n["receivers"]):
                start = np.int64(nid)
                break
        for i in self.inputs:
            iid = i["name"]
            ##print(np.array(i["receivers"]))
            if link_id == np.array(i["receivers"]):
                start = iid
                break
        if not start or not stop:
            print("link %d is hanging down" % link_id)
        return (start, stop)
    
    def connect(self, from_id, target_id)-> bool:
        if from_id  not in self:
            print("from object %d not found" % from_id)
            return False
        if target_id  not in self:
            print("target object %d not found" % from_id)
            return False
        
        index = self.findLink(from_id)
        if index >= 0:
            self.links[index]['receiverId'] = np.int64(target_id)
            print("linked L[%d] to %d" % (from_id, target_id ))
            return True
        index = self.findNeuron(from_id)
        if index >= 0:
            if np.int64(target_id) not in self.neurons[index]["receivers"]:
                self.neurons[index]["receivers"].append(np.int64(target_id))
            print("linked N[%d] to %d" % (from_id, target_id))
            return True
        
        ## id can be name
        index = self.findInput(from_id)
        if index >= 0:
            if np.int64(target_id) not in self.inputs[index]["receivers"]:
                self.inputs[index]["receivers"].append(np.int64(target_id))
            print("linked I[%d] to %d" % (from_id, target_id))
            return True 
        print("source object [%s] not found!" % str(from_id))
        return False
    
pkg = Package()
pkg.loadFile('data/namespacetest_rev0.pkg')
pkg.counts()


pkg ok
inputs:9, outputs:10, neurons:10, links:10, total:39


In [82]:
def showGraph(pkg):
    #net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed =True)

    ZH = "ZERO_HOLE"
    def hasNode(nodes, key)->bool:
        for node in nx_graph.nodes:
            if str(node) == str(key):
                return True
        return False
    nx_graph = nx.DiGraph()

    for n in pkg.neurons:
        nx_graph.add_node(str(n["id"]), size = 10 + int(float(n["currentEnergy"])), title=('neuron %d' % n["id"]), group=1)
    for o in pkg.outputs:
        nx_graph.add_node(str(o["id"]), size = 10, title=('outputs %d' % n["id"]), group=2)
    for i in pkg.inputs:
        nx_graph.add_node(str(i["name"]), size = 5, title=('input %s' % i["name"]), group=3)

    #make direct links
    nx_graph.add_node(str(ZH), size = 1, title=("missed node"), group=1)
    l = pkg.links[1]
    start, stop = pkg.linkEnds(l)
    #print("link ", start, " to " , stop)
    #nx_graph.add_edge(str(start), str(stop))

    for l in pkg.links:
        print(l)
        start, stop = pkg.linkEnds(l)
        link_len = int(l['length'])
        print("link ", start, " to " , str(stop), " sz:", link_len)
        if not start:
            continue
            nx_graph.add_edge(ZH, stop, length = (link_len + 1), weight=(link_len + 1))
        if not hasNode(nx_graph.nodes, str(stop)):
            continue
            nx_graph.add_edge(ZH, stop, value = (link_len + 1), weight=(link_len + 1))
        nx_graph.add_edge(str(start), str(stop), length = (link_len + 1), weight=(link_len + 1) )

    for n in pkg.neurons:
        for rc in n['receivers']:
            if not pkg.isLink(rc):
                nx_graph.add_edge(str(n["id"]), str(rc), length = 1, weight = 1)
    nt.from_nx(nx_graph)
    return nt.show('nx.html')

In [83]:
showGraph(pkg)

{'attenuationPerTick': '0.020000', 'events': [], 'id': 10, 'length': 2, 'receiverId': 64}
link 10 is hanging down
link  0  to  64  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 11, 'length': 2, 'receiverId': 65}
link  Alias1  to  65  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 12, 'length': 2, 'receiverId': 66}
link  Alias2  to  66  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 13, 'length': 2, 'receiverId': 67}
link  Alias3  to  67  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 14, 'length': 2, 'receiverId': 68}
link  Alias4  to  68  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 15, 'length': 2, 'receiverId': 69}
link  Alias5  to  69  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 16, 'length': 2, 'receiverId': 70}
link  Alias6  to  70  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 17, 'length': 2, 'receiverId': 71}
link  Alias7  to  71  sz: 2
{'attenuationPerTick': '0.020000', 'e

In [84]:
## SAVELOAD TEST
saved_data = pkg.dump()
dump_fname = "data/TEST_out.json"
pkg.saveToFile(dump_fname)
#pkg.saveToFile()
saved_data

restored = pkg.loadFile(dump_fname)
showGraph(restored)

pkg ok


AttributeError: 'NoneType' object has no attribute 'neurons'

In [ ]:
p = pkg.clone()
n1 = ObjectFactory.makeNeuron(peak = 10.0, receivers = [], leak = 0.1, currentEnergy = 3.0, id = p.nextId())
p.neurons.append(n1)
p.connect(n1["id"], 64)
p.connect(n1["id"], 65)
p.connect(n1["id"], 69)
g = showGraph(p)
print("new object",n1["id"])
g